In [ ]:
!pip install -q espnet==0.10.0
!pip install -q espnet_model_zoo

In [ ]:
lang = 'en'
fs = 16000 #@param {type:"integer"}
tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [ ]:
!pip3 install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio===0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html


In [ ]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cpu",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [ ]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz
!tar -xf test-clean.tar.gz

--2021-12-08 04:25:49--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/test-clean.tar.gz [following]
--2021-12-08 04:25:50--  http://us.openslr.org/resources/12/test-clean.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M  11.0MB/s    in 19s     

2021-12-08 04:26:09 (17.4 MB/s) - ‘test-clean.tar.gz’ saved [346663984/346663984]



In [ ]:
ls LibriSpeech/

BOOKS.TXT  CHAPTERS.TXT  LICENSE.TXT  README.TXT  SPEAKERS.TXT  test-clean/


In [ ]:
import soundfile as sf
import os
import tensorflow as tf

REQUIRED_SAMPLE_RATE = 16000
SPLIT = "test-clean"

def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

In [ ]:
def fetch_sound_text_mapping():
  flac_files = tf.io.gfile.glob(f"LibriSpeech/{SPLIT}/*/*/*.flac")
  txt_files = tf.io.gfile.glob(f"LibriSpeech/{SPLIT}/*/*/*.txt")

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  file_ids = set(speech_samples.keys()) & set(txt_samples.keys())
  print(f"{len(file_ids)} files are found in LibriSpeech/{SPLIT}")
  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in file_ids]
  return samples

In [ ]:
samples = fetch_sound_text_mapping()

2618 files are found in LibriSpeech/test-clean


In [ ]:
len(samples)

2618

In [ ]:
audio_data = []
text_data = []
transcription = []
for i in range(len(samples)):
  sample = samples[i][0]
  audio_data.append(sample)
  ans = speech2text(sample)
  text, *_ = ans[0]
  transcription.append(text_normalizer(text))
  text_data.append(samples[i][1])

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/espnet2/layers/stft.py:109: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.win_length) // self.hop_length + 1
/usr/local/lib/python3.7/dist-packages/espnet/nets/ctc_prefix_score.py:326: RuntimeWarning: invalid value encountered in logaddexp
  r_prev[:, 0], r_prev[:, 1]


In [ ]:
len(transcription)

2618

In [ ]:
transcription

['IN DETERMINING WHETHER 2 OR MORE ALLIED FORMS OUGHT TO BE RANKED AS SPECIES OR VARIETIES ITS A SURFACTANTLY BUT THATS THE PHYSIOLOGICALLY WHETHER THEYRE A PHYSIOLOGICAL IMPORTANCE BUT MORE IMPORTANTLY WHETHER THEY ARE CONSTANT',
 'THEN HEAVING A HEAVIER SILHOUETTE ID SAY IN THE LAST HES GOING TO HAVE A DREW AND HES GOING TO HAVE TO HEY HES GOT A LOT OF ABANDONED SO ITS NOT  I WOULDNT  I WOULDNT SAY ITS A  BUT ITS NOT AS EASY TO DEAL WITH AN INDIAN AS YOU WOULD WITH THE PHILIPS',
 'THE WORLD WAS WATERED IN MISSES',
 'IT IS NOTABLE THAT THE INDIAN TRIBES HAVE GENERALLY REGARDED THE RELIGION OF THE LATAMS SYNCS WITH FAVORITES SEEING IN THE BOOK OF MOM AND STRIKING AGREEMENT WITH THEIR OWN TRADITIONS',
 'I DID NOT GO SO FAR AS THAT BUT OF THE 3 HE IS PERHAPS THE LEAST UNLIKELY',
 'AND SHE WAS VERY FOND OF YOU TOO AT RACHEL',
 'THE VARIABILITY OF MULTIPLE PARTS',
 'SOME MISCELLANEOUS FORCE SEEMED TO HAVE BROUGHT ABOUT A CONVENTION OF THE ELEMENTS',
 'SHED ALMOST FORGOTTEN THAT IT WAS HERE

In [ ]:
!pip install jiwer
import jiwer
from jiwer import wer


     |████████████████████████████████| 50 kB 3.3 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149858 sha256=df747789b7cb97b64aa8d43b7b1fed74797ddbb7b8ad56285cff56e8eae70603
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
##WER


In [ ]:
total_wer = 0
for i in range(len(transcription)):
  total_wer += wer(transcription[i],text_data[i])

In [ ]:
total_wer/len(transcription)

0.2951883955092679